In [1]:
%matplotlib inline
import os
#import tempfile

import camb
import cobaya
import matplotlib as mpl

import matplotlib.pyplot as plt
import numpy as np

print("     Numpy :", np.__version__)
print("Matplotlib :", mpl.__version__)
print("      CAMB :", camb.__version__)
print("    Cobaya :", cobaya.__version__)

     Numpy : 1.26.4
Matplotlib : 3.10.1
      CAMB : 1.5.9
    Cobaya : 3.5.6


In [2]:
# Fiducial cosmo params. When used in a MCMC, the smooth data must agree with this input values

# set neutrino masses
m1=0
m2=np.sqrt(7.5e-5)
m3=np.sqrt(2.5e-3)

sumnu=m1+m2+m3

f1=m1/sumnu
f2=m2/sumnu
f3=m3/sumnu

print('sumnu =', sumnu)
print('')
print('f1 =', f1, '\nf2 =', f2, '\nf3 =', f3)

cosmo_params = {
    "cosmomc_theta": 0.0104090,
    "logA": {"value": 3.045, "drop": True},
    "As": {"value": "lambda logA: 1e-10*np.exp(logA)"},
    "ombh2": 0.02236,
    "omch2": 0.1202,
    "ns": 0.9649,
    "Alens": 1.0,
    "tau": 0.0544,
    "mnu": sumnu,
    "nnu": 3.044,
}

common_nuis_params = {
    "T_effd": 19.6,
    "beta_d": 1.5,
    "beta_s": -2.5,
    "alpha_s": 1,
    "bandint_shift_LAT_93": 0, # only ideal values for now
    "bandint_shift_LAT_145": 0,
    "bandint_shift_LAT_225": 0,
    "cal_LAT_93": 1,
    "cal_LAT_145": 1,
    "cal_LAT_225": 1,
    "calG_all": 1,
    "alpha_LAT_93": 0,
    "alpha_LAT_145": 0,
    "alpha_LAT_225": 0,
}

TT_nuis_params = {
    "a_tSZ": 3.30,
    "a_kSZ": 1.60,
    "a_p": 6.90,
    "beta_p": 2.08,
    "a_c": 4.90,
    "beta_c": 2.20,
    "a_s": 3.10,
    "T_d": 9.60,
    "a_gtt": 2.81,
    "xi": 0.20,
    "alpha_dT": -0.6,
    "alpha_p": 1,
    "alpha_tSZ": 0.,
    "calT_LAT_93": 1,
    "calT_LAT_145": 1,
    "calT_LAT_225": 1,
}

TE_nuis_params = {
    "a_gte": 0.10,
    "a_pste": 0,
    "alpha_dE": -0.4,
}

EE_nuis_params = {
    "a_gee": 0.10,
    "a_psee": 0,
    "alpha_dE": -0.4,
    "calE_LAT_93": 1,
    "calE_LAT_145": 1,
    "calE_LAT_225": 1,
}

sumnu = 0.05866025403784439

f1 = 0.0 
f2 = 0.14763410387308012 
f3 = 0.8523658961269198


In [3]:
packages_path = "/Users/bradamante/Library/CloudStorage/GoogleDrive-grbmtn@unife.it/My Drive/Work/Projects/Open/cobaya_packages/"

In [4]:
namedir=packages_path+"data/soliket_mflike/smooth_data/split_mflike/"
pre = "data_sacc_"
mflike_options = {"data_folder": namedir,
                  "input_file": pre + "smooth_00000.fits",
                  "cov_Bbl_file": packages_path+"data/MFLike/v0.8/data_sacc_w_covar_and_Bbl.fits",
                  "defaults": {
                        "polarizations": ["TT", "TE", "ET", "EE"],
                        "scales": {
                            "TT": [30, 9000],
                            "TE": [30, 9000],
                            "ET": [30, 9000],
                            "EE": [30, 9000],
                        },
                        "symmetrize": False,
                    },
                 }

lensing_options = {
    "data_filename": "soliket_lensing/smooth_data/clkk_reconstruction_sim_smooth.fits",
    "theory_lmax": 5000,
    "stop_at_error": True}

info_multi = {"likelihood": {
        "soliket.gaussian.MultiGaussianLikelihood": {
            "components": ["mflike.TTTEEE", "soliket.LensingLikelihood"],
            "options": [mflike_options, lensing_options],
            "stop_at_error": True,
        }
    },
    "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                       "WantTransfer": True,
                                       "Transfer.high_precision": True,
                                       "Transfer.kmax": 1.2,
                                       "max_l": 9151,
                                       "max_eta_k": 22877.5,
                                       #"max_l" : 5150,
                                       #"max_eta_k" : 18000.0,
                                       "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3 computed according to oscilation measurements
                                       "num_nu_massive": 2, 
                                       "nu_mass_eigenstates": 2, 
                                       "nu_mass_fractions": [f2,f3],
                                       "nu_mass_numbers": [1,1],
                                       "share_delta_neff": False},
                        "stop_at_error": True}},
    "params": cosmo_params | common_nuis_params | TT_nuis_params | TE_nuis_params | EE_nuis_params,
    "packages_path": packages_path,
    "debug": True,
}
info_multi["theory"]["mflike.BandpowerForeground"] = {'stop_at_error': True}

In [5]:
from cobaya.model import get_model

model_multi = get_model(info_multi)

 2025-05-07 11:21:16,181 [input] 'mflike.BandpowerForeground' could not be found as internal, trying external.
 2025-05-07 11:21:16,459 [root] Numba not available, install it for better performance.
 2025-05-07 11:21:16,470 [input] 'soliket.gaussian.MultiGaussianLikelihood' could not be found as internal, trying external.
 2025-05-07 11:21:16,522 [input] 'mflike.TTTEEE' could not be found as internal component.
 2025-05-07 11:21:16,522 [input] 'mflike.TTTEEE' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 11:21:16,522 [input] 'mflike.TTTEEE' could not be found as internal component.
 2025-05-07 11:21:16,522 [input] 'mflike.TTTEEE' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 11:21:16,523 [input] 'mflike.TTTEEE' could not be found as internal component.
 2025-05-07 11:21:16,523 [input] 'mflike.TTTEEE' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 1

In [6]:
model_multi.loglikes()

(array([-1694.84350768]), [2.101003120287951e-09])

In [7]:
model_multi.components

[soliket.gaussian.MultiGaussianLikelihood,
 camb.transfers,
 camb,
 mflike.BandpowerForeground]

In [8]:
Dl = model_multi.components[2].get_Cl(ell_factor=True)
fg_totals = model_multi.components[3].get_fg_totals()

In [9]:
params = {**cosmo_params,**common_nuis_params,**TT_nuis_params,**EE_nuis_params,**TE_nuis_params}

In [10]:
mflike = model_multi.components[0].likelihoods[0]
dls = {s: Dl[s][mflike.l_bpws] for s, _ in mflike.lcuts.items()}
#combine CMB and FG and add svstematics
DlsObs = mflike.get_modified_theory(dls, fg_totals, **params)

In [11]:
ps_dic = {}
ps_vec = np.zeros_like(mflike.data_vec)

for m in mflike.spec_meta:
    p = m["pol"]
    ids = m["ids"]
    w = m["bpw"]
    t1 = m["t1"]
    t2 = m["t2"]
    
    #print(p, ids, t1, t2)
    if (t1+"x"+t2 not in ps_dic.keys ()):
        ps_dic[t1+"x"+t2]={"lbin": m["leff"]}
        
    dls_obs = DlsObs[p, m["t2"], m["t1"]] if m["hasYX_xsp"] else DlsObs[p, m["t1"], m["t2"]]
    for i, nonzero, weights in zip (m["ids"], w.nonzeros, w.sliced_weights):
        ps_vec[i] = weights @ dls_obs[nonzero]
        
    ps_dic[t1+"x"+t2].update({p: ps_vec[ids]})

In [12]:
ps_dic.keys() #check that all spectra are here

dict_keys(['LAT_93xLAT_93', 'LAT_93xLAT_145', 'LAT_93xLAT_225', 'LAT_145xLAT_145', 'LAT_145xLAT_225', 'LAT_225xLAT_225'])

In [13]:
namedir=packages_path+"/data/soliket_mflike/smooth_data/split_mflike/"
#namedir="/Users/bradamante/Google Drive/My Drive/Work/Projects/Open/cobaya_packages/data/soliket_mflike/smooth_data/"
for k in ps_dic.keys():
    namefile = "Dl_frommulti_"+k+"_auto_00000.dat"
    l = ps_dic[k]["lbin"]
    tt = ps_dic[k]["tt"]
    te = ps_dic[k]["te"]
    ee = ps_dic[k]["ee"]
    tbebbb = np.zeros(len(l))
    np.savetxt(namedir+namefile,np.column_stack((l,tt,te,tbebbb,te,tbebbb,ee,tbebbb,tbebbb,tbebbb)))

In [14]:
cltest=np.loadtxt(namedir+namefile)
cltest.shape

(147, 10)

In [15]:
data = {}
sim_suffix = "00000"
for spec_name in ps_dic.keys():
    na, nb = spec_name.split("x")
    data[na,nb] = {}
    spec = np.loadtxt("%s/Dl_frommulti_%s_auto_%s.dat" % (namedir, spec_name, sim_suffix), unpack=True)
    ps = {"lbin": spec[0],
          "TT": spec[1],
          "TE": spec[2],
          "TB": spec[3],
          "ET": spec[4],
          "BT": spec[5],
          "EE": spec[6],
          "EB": spec[7],
          "BE": spec[8],
          "BB": spec[9]}
    data[na,nb] = ps

In [16]:
exp_freq = ['LAT_93', 'LAT_145', 'LAT_225']
pols = ["T", "E", "B"]
map_types = {"T": "0", "E": "e", "B": "b"}

def get_x_iterator():
    for id_efa, efa in enumerate(exp_freq):
        for id_efb, efb in enumerate(exp_freq):
            if (id_efa > id_efb): continue
            for ipa, pa in enumerate(pols):
                if (efa == efb):
                    polsb = pols[ipa:]
                else:
                    polsb = pols
                for pb in polsb:
                    yield (efa, efb, pa, pb)
                    print(efa, efb, pa, pb)

In [17]:
import sacc

spec_sacc =  sacc.Sacc()

In [18]:
for exp_f in exp_freq:
    print("%s_s0" % (exp_f))

    my_data_bandpasses = {"nu":np.array([float(exp_f.split("_")[1])]), "b_nu":np.array([1.])}
    my_data_beams = {"l":np.arange(10000), "bl":np.ones(10000)}

    # CMB temperature 
    spec_sacc.add_tracer("NuMap", "%s_s0" % (exp_f),
                                 quantity="cmb_temperature", spin=0,
                                 nu=mflike.bands[exp_f+"_s0"]["nu"],
                                 bandpass=mflike.bands[exp_f+"_s0"]["bandpass"],
                                 ell=my_data_beams["l"],
                                 beam=my_data_beams["bl"])
    
    # CMB polarization
    spec_sacc.add_tracer("NuMap", "%s_s2" % (exp_f),
                                 quantity="cmb_polarization", spin=2,
                                 nu=mflike.bands[exp_f+"_s2"]["nu"],
                                 bandpass=mflike.bands[exp_f+"_s2"]["bandpass"],
                                 ell=my_data_beams["l"],
                                 beam=my_data_beams["bl"])

LAT_93_s0
LAT_145_s0
LAT_225_s0


In [19]:
for id_x, (efa, efb, pa, pb) in enumerate(get_x_iterator()):
    if pa == "T":
        ta_name = "%s_s0" % (efa)
    else:
        ta_name = "%s_s2" % (efa)

    if pb == "T":
        tb_name = "%s_s0" % (efb)
    else:
        tb_name = "%s_s2" % (efb)

    if pb == "T":
        cl_type = "cl_" + map_types[pb] + map_types[pa]
    else:
        cl_type = "cl_" + map_types[pa] + map_types[pb]

    lbin = data[efa, efb]["lbin"]
    cb = data[efa, efb][pa + pb]

    spec_sacc.add_ell_cl(cl_type, ta_name, tb_name, lbin, cb)

LAT_93 LAT_93 T T
LAT_93 LAT_93 T E
LAT_93 LAT_93 T B
LAT_93 LAT_93 E E
LAT_93 LAT_93 E B
LAT_93 LAT_93 B B
LAT_93 LAT_145 T T
LAT_93 LAT_145 T E
LAT_93 LAT_145 T B
LAT_93 LAT_145 E T
LAT_93 LAT_145 E E
LAT_93 LAT_145 E B
LAT_93 LAT_145 B T
LAT_93 LAT_145 B E
LAT_93 LAT_145 B B
LAT_93 LAT_225 T T
LAT_93 LAT_225 T E
LAT_93 LAT_225 T B
LAT_93 LAT_225 E T
LAT_93 LAT_225 E E
LAT_93 LAT_225 E B
LAT_93 LAT_225 B T
LAT_93 LAT_225 B E
LAT_93 LAT_225 B B
LAT_145 LAT_145 T T
LAT_145 LAT_145 T E
LAT_145 LAT_145 T B
LAT_145 LAT_145 E E
LAT_145 LAT_145 E B
LAT_145 LAT_145 B B
LAT_145 LAT_225 T T
LAT_145 LAT_225 T E
LAT_145 LAT_225 T B
LAT_145 LAT_225 E T
LAT_145 LAT_225 E E
LAT_145 LAT_225 E B
LAT_145 LAT_225 B T
LAT_145 LAT_225 B E
LAT_145 LAT_225 B B
LAT_225 LAT_225 T T
LAT_225 LAT_225 T E
LAT_225 LAT_225 T B
LAT_225 LAT_225 E E
LAT_225 LAT_225 E B
LAT_225 LAT_225 B B


In [20]:
spec_sacc.save_fits("%s/data_sacc_smooth_%s.fits" % (namedir, sim_suffix), overwrite=True)

In [21]:
from astropy.io import fits

fname = packages_path+"data/LensingLikelihood/clkk_reconstruction_sim.fits"
hdul = fits.open(fname)

hdul.info()

Filename: /Users/bradamante/Library/CloudStorage/GoogleDrive-grbmtn@unife.it/My Drive/Work/Projects/Open/cobaya_packages/data/LensingLikelihood/clkk_reconstruction_sim.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   ()      
  1  window:TopHat    1 BinTableHDU     17   0R x 3C   [D, D, D]   
  2  window:LogTopHat    1 BinTableHDU     17   0R x 3C   [D, D, D]   
  3  window:Bandpower    1 BinTableHDU     17   3000R x 2C   [K, 19D]   
  4  tracer:Misc    1 BinTableHDU     15   0R x 2C   [D, D]   
  5  tracer:Misc    1 BinTableHDU     15   0R x 2C   [D, D]   
  6  tracer:Map:ck:beam    1 BinTableHDU     19   3000R x 2C   [K, D]   
  7  tracer:Map:ck:beam    1 BinTableHDU     19   3000R x 2C   [K, D]   
  8  data:cl_00    1 BinTableHDU     24   19R x 6C   [2A, 2A, D, K, D, K]   
  9  covariance    1 ImageHDU        11   (19, 19)   float64   


In [22]:
lensing = model_multi.components[0].likelihoods[1]

In [23]:
hdul[8].data["value"][:] = lensing._get_theory(**cosmo_params)

In [24]:
ndir = packages_path+"data/LensingLikelihood/soliket_lensing/smooth_data/clkk_reconstruction_sim_frommulti_smooth.fits"
hdul.writeto(ndir, overwrite=True)

# Test new data

In [25]:
namedir=packages_path+"data/soliket_mflike/smooth_data/split_mflike/"
pre = "data_sacc_"
mflike_options = {"data_folder": namedir,
                  "input_file": pre + "smooth_00000.fits",
                  "cov_Bbl_file": packages_path+"data/MFLike/v0.8/data_sacc_w_covar_and_Bbl.fits",
                  "defaults": {
                        "polarizations": ["TT", "TE", "ET", "EE"],
                        "scales": {
                            "TT": [30, 9000],
                            "TE": [30, 9000],
                            "ET": [30, 9000],
                            "EE": [30, 9000],
                        },
                        "symmetrize": False,
                    },
                 }

lensing_options = {
    "data_filename": "soliket_lensing/smooth_data/clkk_reconstruction_sim_frommulti_smooth.fits",
    "theory_lmax": 5000,
    "stop_at_error": True}

info_multi = {"likelihood": {
        "soliket.gaussian.MultiGaussianLikelihood": {
            "components": ["mflike.TTTEEE", "soliket.LensingLikelihood"],
            "options": [mflike_options, lensing_options],
            "stop_at_error": True,
        }
    },
    "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                       "WantTransfer": True,
                                       "Transfer.high_precision": True,
                                       "Transfer.kmax": 1.2,
                                       "max_l": 9151,
                                       "max_eta_k": 22877.5,
                                       #"max_l" : 5150,
                                       #"max_eta_k" : 18000.0,
                                       "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3 computed according to oscilation measurements
                                       "num_nu_massive": 2, 
                                       "nu_mass_eigenstates": 2, 
                                       "nu_mass_fractions": [f2,f3],
                                       "nu_mass_numbers": [1,1],
                                       "share_delta_neff": False},
                        "stop_at_error": True}},
    "params": cosmo_params | common_nuis_params | TT_nuis_params | TE_nuis_params | EE_nuis_params,
    "packages_path": packages_path,
    "debug": True,
}
info_multi["theory"]["mflike.BandpowerForeground"] = {'stop_at_error': True}

In [26]:
from cobaya.model import get_model

model_multi = get_model(info_multi)
model_multi.loglikes()

 2025-05-07 11:22:06,042 [input] 'mflike.BandpowerForeground' could not be found as internal, trying external.
 2025-05-07 11:22:06,058 [input] 'soliket.gaussian.MultiGaussianLikelihood' could not be found as internal, trying external.
 2025-05-07 11:22:06,059 [input] 'mflike.TTTEEE' could not be found as internal component.
 2025-05-07 11:22:06,059 [input] 'mflike.TTTEEE' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 11:22:06,060 [input] 'mflike.TTTEEE' could not be found as internal component.
 2025-05-07 11:22:06,061 [input] 'mflike.TTTEEE' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 11:22:06,061 [input] 'mflike.TTTEEE' could not be found as internal component.
 2025-05-07 11:22:06,061 [input] 'mflike.TTTEEE' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 11:22:06,071 [input] 'soliket.LensingLikelihood' could not be found as internal component

(array([-1694.84350768]), [2.101003120287951e-09])

check if mflike chi2 is zero

In [27]:
params = {**cosmo_params,**common_nuis_params,**TT_nuis_params,**EE_nuis_params,**TE_nuis_params}
model_multi.components[0].likelihoods[0].logp(**params)-model_multi.components[0].likelihoods[0].logp_const

0.0

check if lensing chi2 is zero

In [28]:
lensing=model_multi.components[0].likelihoods[1]

In [29]:
lensing._get_theory(**cosmo_params)-lensing._get_data()[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [30]:
delta = lensing._get_theory(**cosmo_params)-lensing._get_data()[1]
-0.5 * np.dot(delta, np.linalg.inv(lensing._get_cov()).dot(delta))

-0.0

# Test TT only case

In [44]:
common_nuis_params = {
    "T_effd": 19.6,
    "beta_d": 1.5,
    "beta_s": -2.5,
    "alpha_s": 1,
    "bandint_shift_LAT_93": 0,
    "bandint_shift_LAT_145": 0,
    "bandint_shift_LAT_225": 0,
    "cal_LAT_93": 1,
    "cal_LAT_145": 1,
    "cal_LAT_225": 1,
    "calG_all": 1,
    "alpha_LAT_93": 0,
    "alpha_LAT_145": 0,
    "alpha_LAT_225": 0,
}

TT_nuis_params = {
    "a_tSZ": 3.30,
    "a_kSZ": 1.60,
    "a_p": 6.90,
    "beta_p": 2.20,
    "a_c": 4.90,
    "beta_c": 2.20,
    "a_s": 3.10,
    "T_d": 9.60,
    "a_gtt": 2.80,
    "xi": 0.10,
    "alpha_dT": -0.6,
    "alpha_p": 1,
    "alpha_tSZ": 0.,
    "calT_LAT_93": 1,
    "calT_LAT_145": 1,
    "calT_LAT_225": 1,
}

nuis_params = common_nuis_params | TT_nuis_params

namedir=packages_path+"data/MFLike/v0.8/"
pre = "LAT_simu_sacc_"
mflike_options = {"data_folder": namedir,
                  "input_file": pre + "00000.fits",
                  "cov_Bbl_file": packages_path+"data/MFLike/v0.8/data_sacc_w_covar_and_Bbl.fits",
                  #"data": {
                  #      "experiments": ["LAT_93"],
                  #      "spectra": [{"experiments": ["LAT_93", "LAT_93"]}]
                  #  }
                  
                 }

lensing_options = {
    "data_filename": "soliket_lensing/smooth_data/clkk_reconstruction_sim_smooth.fits",
    "theory_lmax": 5000,
    "stop_at_error": True}

info_multi_TT = {"likelihood": {
        "soliket.gaussian.MultiGaussianLikelihood": {
            "components": ["mflike.TT", "soliket.LensingLikelihood"],
            "options": [mflike_options, lensing_options],
            "stop_at_error": True,
        }
    },
    "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                      "WantTransfer": True,
                                      "lmax": 9001,
                                      "Transfer.high_precision": True,
                                      "Transfer.kmax": 1.2,
                                      "max_l": 9701,
                                      "max_eta_k": 24252.5,
                                      "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3 computed according to oscilation measurements
                                      "num_nu_massive": 2, 
                                      "nu_mass_eigenstates": 2, 
                                      "nu_mass_fractions": [f2,f3],
                                      "nu_mass_numbers": [1,1],
                                      "share_delta_neff": False
                                      },
                        "stop_at_error": True}},
    "params": cosmo_params | nuis_params,
    "packages_path": packages_path,
    "debug": True,
}
info_multi_TT["theory"]["mflike.BandpowerForeground"] = {"requested_cls": ['tt'],
                                                         'stop_at_error': True}

In [45]:
from cobaya.model import get_model

model_multi_TT = get_model(info_multi_TT)

 2025-05-07 11:37:54,484 [input] 'mflike.BandpowerForeground' could not be found as internal, trying external.
 2025-05-07 11:37:54,498 [input] 'soliket.gaussian.MultiGaussianLikelihood' could not be found as internal, trying external.
 2025-05-07 11:37:54,506 [input] 'mflike.TT' could not be found as internal component.
 2025-05-07 11:37:54,511 [input] 'mflike.TT' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 11:37:54,518 [input] 'mflike.TT' could not be found as internal component.
 2025-05-07 11:37:54,521 [input] 'mflike.TT' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 11:37:54,522 [input] 'mflike.TT' could not be found as internal component.
 2025-05-07 11:37:54,525 [input] 'mflike.TT' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 11:37:54,536 [input] 'soliket.LensingLikelihood' could not be found as internal component.
 2025-05-07 11:37:54,5

In [46]:
model_multi_TT.loglikes()
TT=model_multi_TT.components[0].likelihoods[0]

In [47]:
params={**cosmo_params,**nuis_params}
TT.logp(**params)

-6010.945673048354

In [48]:
namedir=packages_path+"data/MFLike/v0.8/"
pre = "LAT_simu_sacc_"
info = {
    "likelihood": {
    "mflike.TT": {
        "package_install": "pip",
        "data_folder": namedir,
        "input_file": pre + "00000.fits",
        "cov_Bbl_file": packages_path + "/data/MFLike/v0.8/data_sacc_w_covar_and_Bbl.fits",
    }
        
},
        "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                           "WantTransfer": True,
                                           "Transfer.high_precision": True,
                                           "Transfer.kmax": 1.2,
                                           "max_l": 9701,
                                           "max_eta_k": 24252.5,
                                           "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3
                                           "num_nu_massive": 2,
                                           "nu_mass_eigenstates": 2,
                                           "nu_mass_fractions": [f2,f3],
                                           "nu_mass_numbers": [1,1],
                                           "share_delta_neff": False},
                            "stop_at_error": True}},
        "params": cosmo_params | common_nuis_params | TT_nuis_params,
        "packages_path": packages_path,
        "debug": True,
}

info["theory"]["mflike.BandpowerForeground"] = {'requested_cls': ['tt'],
                                                    'stop_at_error': True
                                                  }

from cobaya.model import get_model

model_tt = get_model(info)
mflike_tt = model_tt.likelihood["mflike.TT"]

 2025-05-07 11:38:37,714 [input] 'mflike.BandpowerForeground' could not be found as internal, trying external.
 2025-05-07 11:38:37,726 [input] 'mflike.TT' could not be found as internal, trying external.
 2025-05-07 11:38:37,740 [model] Input info updated with defaults (dumped to YAML):
theory:
  camb:
    version: null
    path: null
    speed: 0.3
    stop_at_error: true
    extra_args:
      lens_potential_accuracy: 1
      WantTransfer: true
      Transfer.high_precision: true
      Transfer.kmax: 1.2
      max_l: 9701
      max_eta_k: 24252.5
      num_nu_massless: 1.044
      num_nu_massive: 2
      nu_mass_eigenstates: 2
      nu_mass_fractions:
      - 0.14763410387308012
      - 0.8523658961269198
      nu_mass_numbers:
      - 1
      - 1
      share_delta_neff: false
    ignore_obsolete: false
    use_renames: false
    external_primordial_pk: false
    renames:
      omegabh2: ombh2
      omegach2: omch2
      omegal: omega_de
      omegak: omk
      yhe: YHe
      yheused

In [49]:
model_tt.loglikes()
mflike_tt.logp(**params)

 2025-05-07 11:38:39,638 [model] Got input parameters: {'cosmomc_theta': 0.010409, 'logA': 3.045, 'As': 2.101003120287951e-09, 'ombh2': 0.02236, 'omch2': 0.1202, 'ns': 0.9649, 'Alens': 1.0, 'tau': 0.0544, 'mnu': 0.05866025403784439, 'nnu': 3.044, 'T_effd': 19.6, 'beta_d': 1.5, 'beta_s': -2.5, 'alpha_s': 1.0, 'bandint_shift_LAT_93': 0.0, 'bandint_shift_LAT_145': 0.0, 'bandint_shift_LAT_225': 0.0, 'cal_LAT_93': 1.0, 'cal_LAT_145': 1.0, 'cal_LAT_225': 1.0, 'calG_all': 1.0, 'alpha_LAT_93': 0.0, 'alpha_LAT_145': 0.0, 'alpha_LAT_225': 0.0, 'a_tSZ': 3.3, 'a_kSZ': 1.6, 'a_p': 6.9, 'beta_p': 2.2, 'a_c': 4.9, 'beta_c': 2.2, 'a_s': 3.1, 'T_d': 9.6, 'a_gtt': 2.8, 'xi': 0.1, 'alpha_dT': -0.6, 'alpha_p': 1.0, 'alpha_tSZ': 0.0, 'calT_LAT_93': 1.0, 'calT_LAT_145': 1.0, 'calT_LAT_225': 1.0}
 2025-05-07 11:38:39,642 [camb.transfers] Got parameters {'cosmomc_theta': 0.010409, 'ombh2': 0.02236, 'omch2': 0.1202, 'Alens': 1.0, 'tau': 0.0544, 'mnu': 0.05866025403784439, 'nnu': 3.044}
 2025-05-07 11:38:39,643

-6010.945673048354

In [50]:
mflike_tt.logp(**params)-TT.logp(**params)

 2025-05-07 11:39:13,589 [mflike.tt] Log-likelihood value computed = -6010.945673048354 (Χ² = -18879.59643513606)
 2025-05-07 11:39:13,599 [mflike.tt] Log-likelihood value computed = -6010.945673048354 (Χ² = -18879.59643513606)


0.0

# Test with single channel

In [51]:
common_nuis_params_93 = {
    "T_effd": 19.6,
    "beta_d": 1.5,
    "beta_s": -2.5,
    "alpha_s": 1,
    "bandint_shift_LAT_93": 0, # only ideal values for now
    #"bandint_shift_LAT_145": 0,
    #"bandint_shift_LAT_225": 0,
    "cal_LAT_93": 1,
    #"cal_LAT_145": 1,
    #"cal_LAT_225": 1,
    "calG_all": 1,
    "alpha_LAT_93": 0,
    #"alpha_LAT_145": 0,
    #"alpha_LAT_225": 0,
}

TT_nuis_params_93 = {
    "a_tSZ": 3.30,
    "a_kSZ": 1.60,
    "a_p": 6.90,
    "beta_p": 2.08,
    "a_c": 4.90,
    "beta_c": 2.20,
    "a_s": 3.10,
    "T_d": 9.60,
    "a_gtt": 2.81,
    "xi": 0.20,
    "alpha_dT": -0.6,
    "alpha_p": 1,
    "alpha_tSZ": 0.,
    "calT_LAT_93": 1,
    #"calT_LAT_145": 1,
    #"calT_LAT_225": 1,
}

TE_nuis_params = {
    "a_gte": 0.10,
    "a_pste": 0,
    "alpha_dE": -0.4,
}

EE_nuis_params_93 = {
    "a_gee": 0.10,
    "a_psee": 0,
    "alpha_dE": -0.4,
    "calE_LAT_93": 1,
    #"calE_LAT_145": 1,
    #"calE_LAT_225": 1,
}

In [68]:
lensing_options = {"theory_lmax": 5000}
mflike_options = {
    "input_file": "LAT_simu_sacc_00044.fits",
    "cov_Bbl_file": "data_sacc_w_covar_and_Bbl.fits",
    "stop_at_error": True,
    "data": {
        "experiments": ["LAT_93"],
        "spectra": [
            {"experiments": ["LAT_93", "LAT_93"]},
        ],
    },
}
camb_options = {"extra_args": {"lens_potential_accuracy": 1,
                                           "WantTransfer": True,
                                           "Transfer.high_precision": True,
                                           "Transfer.kmax": 1.2,
                                           "max_l": 9701,
                                           "max_eta_k": 24252.5,
                                           "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3
                                           "num_nu_massive": 2,
                                           "nu_mass_eigenstates": 2,
                                           "nu_mass_fractions": [f2,f3],
                                           "nu_mass_numbers": [1,1],
                                           "share_delta_neff": False},
                            "stop_at_error": True}

mflike_params = cosmo_params | common_nuis_params_93 | TT_nuis_params_93 | TE_nuis_params | EE_nuis_params_93
lensing_params = cosmo_params
info = {
    "likelihood": {
        "soliket.gaussian.MultiGaussianLikelihood": {
            "components": ["mflike.TTTEEE", "soliket.LensingLikelihood"],
            "options": [mflike_options, lensing_options],
            #"components": {
            #    "mflike.TTTEEE": mflike_options,
            #    "soliket.LensingLikelihood": lensing_options,
            #},
            "stop_at_error": True,
        }
    },
    "theory": {
        "camb": camb_options,
        "mflike.BandpowerForeground": {"stop_at_error": True},
    },
    "params": mflike_params,
}
info1 = {
        "likelihood": {"mflike.TTTEEE": mflike_options},
        "theory": {
            "camb": camb_options,
            "mflike.BandpowerForeground": {"stop_at_error": True},
        },
    "params": mflike_params,
}
info2 = {
    "likelihood": {"soliket.LensingLikelihood": lensing_options},
    "theory": {"camb": camb_options},
    "params": lensing_params,
}

In [69]:
from cobaya.model import get_model
model = get_model(info)
model1 = get_model(info1)
model2 = get_model(info2)

 2025-05-07 13:13:45,237 [camb] `camb` module loaded successfully from /Users/bradamante/miniconda/envs/soliket-tests/lib/python3.11/site-packages/camb
 2025-05-07 13:13:46,044 [mflike.ttteee] Number of bins used: 441
 2025-05-07 13:13:46,051 [mflike.ttteee] Initialized!
 2025-05-07 13:13:46,060 [soliket.lensing.lensinglikelihood] Initialising.
 2025-05-07 13:13:46,114 [camb] `camb` module loaded successfully from /Users/bradamante/miniconda/envs/soliket-tests/lib/python3.11/site-packages/camb
 2025-05-07 13:13:53,132 [soliket.gaussian.multigaussianlikelihood] Initialized.
 2025-05-07 13:13:53,171 [camb] `camb` module loaded successfully from /Users/bradamante/miniconda/envs/soliket-tests/lib/python3.11/site-packages/camb
 2025-05-07 13:13:53,568 [mflike.ttteee] Number of bins used: 441
 2025-05-07 13:13:53,570 [mflike.ttteee] Initialized!
 2025-05-07 13:13:53,594 [camb] `camb` module loaded successfully from /Users/bradamante/miniconda/envs/soliket-tests/lib/python3.11/site-packages/c

In [70]:
print(model.loglikes())
print(model1.loglikes())
print(model2.loglikes())
print(model1.loglikes()[0]+model2.loglikes()[0])

(array([-29115.48806725]), [2.101003120287951e-09])
(array([-29352.72859178]), [2.101003120287951e-09])
(array([237.24052453]), [2.101003120287951e-09])
[-29115.48806725]


In [71]:
mflike = model1.likelihood["mflike.TTTEEE"]
lensing = model2.likelihood["soliket.LensingLikelihood"]

In [72]:
print(mflike.logp(**mflike_params)-model.components[0].likelihoods[0].logp(**mflike_params))
print(lensing.logp(**cosmo_params)-model.components[0].likelihoods[1].logp(**cosmo_params))

0.0
0.0


# DEBUG

In [116]:
cosmo_params = {
    "cosmomc_theta": 0.0104090,
    "logA": {"value": 3.045, "drop": True},
    "As": {"value": "lambda logA: 1e-10*np.exp(logA)"},
    "ombh2": 0.02236,
    "omch2": 0.1202,
    "ns": 0.9649,
    "Alens": 1.0,
    "tau": 0.0544,
    "mnu": sumnu,
    "nnu": 3.044,
}

common_nuis_params = {
    "T_effd": 19.6,
    "beta_d": 1.5,
    "beta_s": -2.5,
    "alpha_s": 1,
    "bandint_shift_LAT_93": 0, # only ideal values for now
    "bandint_shift_LAT_145": 0,
    "bandint_shift_LAT_225": 0,
    "cal_LAT_93": 1,
    "cal_LAT_145": 1,
    "cal_LAT_225": 1,
    "calG_all": 1,
    "alpha_LAT_93": 0,
    "alpha_LAT_145": 0,
    "alpha_LAT_225": 0,
}

TT_nuis_params = {
    "a_tSZ": 3.30,
    "a_kSZ": 1.60,
    "a_p": 6.90,
    "beta_p": 2.08,
    "a_c": 4.90,
    "beta_c": 2.20,
    "a_s": 3.10,
    "T_d": 9.60,
    "a_gtt": 2.81,
    "xi": 0.20,
    "alpha_dT": -0.6,
    "alpha_p": 1,
    "alpha_tSZ": 0.,
    "calT_LAT_93": 1,
    "calT_LAT_145": 1,
    "calT_LAT_225": 1,
}

TE_nuis_params = {
    "a_gte": 0.10,
    "a_pste": 0,
    "alpha_dE": -0.4,
}

EE_nuis_params = {
    "a_gee": 0.10,
    "a_psee": 0,
    "alpha_dE": -0.4,
    "calE_LAT_93": 1,
    "calE_LAT_145": 1,
    "calE_LAT_225": 1,
}

namedir=packages_path+"data/soliket_mflike/smooth_data/split_mflike/"
pre = "data_sacc_"
mflike_options = {"data_folder": namedir,
                  "input_file": pre + "smooth_00000.fits",
                  "cov_Bbl_file": packages_path+"data/MFLike/v0.8/data_sacc_w_covar_and_Bbl.fits",
                  "defaults": {
                        "polarizations": ["TT", "TE", "ET", "EE"],
                        "scales": {
                            "TT": [30, 9000],
                            "TE": [30, 9000],
                            "ET": [30, 9000],
                            "EE": [30, 9000],
                        },
                        "symmetrize": False,
                    },
                 }

lensing_options = {
    "data_filename": "soliket_lensing/smooth_data/clkk_reconstruction_sim_smooth.fits",
    "theory_lmax": 5000,
    "stop_at_error": True}

info_multi = {"likelihood": {
        "soliket.gaussian.MultiGaussianLikelihood": {
            "components": ["mflike.TTTEEE", "soliket.LensingLikelihood"],
            "options": [mflike_options, lensing_options],
            "stop_at_error": True,
        }
    },
    "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                       "WantTransfer": True,
                                       "Transfer.high_precision": True,
                                       "Transfer.kmax": 1.2,
                                       #"max_l": 9151,
                                       #"max_eta_k": 22877.5,
                                       #"max_l" : 5150,
                                       #"max_eta_k" : 18000.0,
                                       "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3 computed according to oscilation measurements
                                       "num_nu_massive": 2, 
                                       "nu_mass_eigenstates": 2, 
                                       "nu_mass_fractions": [f2,f3],
                                       "nu_mass_numbers": [1,1],
                                       "share_delta_neff": False},
                        "stop_at_error": True}},

    "params": cosmo_params | common_nuis_params | TT_nuis_params | TE_nuis_params | EE_nuis_params,
    "packages_path": packages_path,
    "debug": True,
}

In [117]:
info_multi["theory"]["mflike.BandpowerForeground"] = {'stop_at_error': True}

In [118]:
from cobaya.model import get_model

model_multi = get_model(info_multi)
my_mflike_multi=model_multi.components[0].likelihoods[0]
lensing_multi = model_multi.components[0].likelihoods[1]

 2025-05-07 14:50:21,783 [input] 'mflike.BandpowerForeground' could not be found as internal, trying external.
 2025-05-07 14:50:21,798 [input] 'soliket.gaussian.MultiGaussianLikelihood' could not be found as internal, trying external.
 2025-05-07 14:50:21,798 [input] 'mflike.TTTEEE' could not be found as internal component.
 2025-05-07 14:50:21,799 [input] 'mflike.TTTEEE' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 14:50:21,801 [input] 'mflike.TTTEEE' could not be found as internal component.
 2025-05-07 14:50:21,801 [input] 'mflike.TTTEEE' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 14:50:21,802 [input] 'mflike.TTTEEE' could not be found as internal component.
 2025-05-07 14:50:21,802 [input] 'mflike.TTTEEE' could not be found. Tried loading internal classes. No component path was given.
 2025-05-07 14:50:21,811 [input] 'soliket.LensingLikelihood' could not be found as internal component

Total loglike

In [119]:
params = {**cosmo_params,**common_nuis_params,**TT_nuis_params,**EE_nuis_params,**TE_nuis_params}
model_multi.loglikes()#params, cached=False)

(array([-1694.85393163]), [2.101003120287951e-09])

input paremeters

In [120]:
model_multi.provider.requirement_providers["CAMB_transfers"].current_state["results"][0]

class: <CAMBparams>
 WantCls = True
 WantTransfer = True
 WantScalars = True
 WantTensors = False
 WantVectors = False
 WantDerivedParameters = True
 Want_cl_2D_array = False
 Want_CMB = True
 Want_CMB_lensing = True
 DoLensing = True
 NonLinear = NonLinear_lens
 Transfer: <TransferParams>
   high_precision = True
   accurate_massive_neutrinos = False
   kmax = 1.2
   k_per_logint = 0
   PK_num_redshifts = 1
   PK_redshifts = [0.0]
 want_zstar = False
 want_zdrag = False
 min_l = 2
 max_l = 9151
 max_l_tensor = 600
 max_eta_k = 22877.5
 max_eta_k_tensor = 1200.0
 ombh2 = 0.02236
 omch2 = 0.1202
 omk = 0.0
 omnuh2 = 0.0006304672808897119
 H0 = 67.26717968167824
 TCMB = 2.7255
 YHe = 0.24585121447518005
 num_nu_massless = 1.044
 num_nu_massive = 2
 nu_mass_eigenstates = 2
 share_delta_neff = False
 nu_mass_degeneracies = [1.0146666666666666, 0.0]
 nu_mass_fractions = [0.14763410387308012, 0.8523658961269198]
 nu_mass_numbers = [1, 1]
 InitPower: <InitialPowerLaw>
   tensor_parameterizati

Check if MFLike chi2 is 0 (we are using smooth data)

In [121]:
#params = {**fg_params,**nuisance_params,**cosmo_params}
my_mflike_multi.logp(**params)-my_mflike_multi.logp_const

0.0

Check if lensing chi2 is 0

In [122]:
delta = lensing_multi._get_theory(**cosmo_params)-lensing_multi._get_data()[1]
-0.5 * np.dot(delta, np.linalg.inv(lensing_multi._get_cov()).dot(delta))

-0.010423954430914316

Check if LensingLikelihood chi2 is 0

This is not. The reason is that the theory does not match the data, i.e., delta above is non-zero.

Why in the multi case the theory computed for lensing does not match the theory computed from lensing alone? Checking for this. We will load LensingLikelihood only and the hand-made combo MFLike+LensingLikelihood to check for CAMBdata params.

In [127]:
info_lensing = {
    "likelihood": {
        "soliket.LensingLikelihood": {
            "data_filename": "soliket_lensing/smooth_data/clkk_reconstruction_sim_smooth.fits",
            "theory_lmax": 5000,
            "stop_at_error": False}
    },
    "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                       "WantTransfer": True,
                                       "Transfer.high_precision": True,
                                       "Transfer.kmax": 1.2,
                                       #"max_l" : 5150,
                                       #"max_eta_k" : 18000.0,
                                       "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3 computed according to oscilation measurements
                                       "num_nu_massive": 2, 
                                       "nu_mass_eigenstates": 2, 
                                       "nu_mass_fractions": [f2,f3],
                                       "nu_mass_numbers": [1,1],
                                       "share_delta_neff": False},
                        "stop_at_error": True}},
    "params": cosmo_params,
    "packages_path": packages_path,
    "debug": True,
}

In [128]:
from cobaya.model import get_model
model_lensing = get_model(info_lensing)

 2025-05-07 14:53:51,478 [input] 'soliket.LensingLikelihood' could not be found as internal, trying external.
 2025-05-07 14:53:51,484 [model] Input info updated with defaults (dumped to YAML):
theory:
  camb:
    version: null
    path: null
    speed: 0.3
    stop_at_error: true
    extra_args:
      lens_potential_accuracy: 1
      WantTransfer: true
      Transfer.high_precision: true
      Transfer.kmax: 1.2
      num_nu_massless: 1.044
      num_nu_massive: 2
      nu_mass_eigenstates: 2
      nu_mass_fractions:
      - 0.14763410387308012
      - 0.8523658961269198
      nu_mass_numbers:
      - 1
      - 1
      share_delta_neff: false
    ignore_obsolete: false
    use_renames: false
    external_primordial_pk: false
    renames:
      omegabh2: ombh2
      omegach2: omch2
      omegal: omega_de
      omegak: omk
      yhe: YHe
      yheused: YHe
      YpBBN: Y_p
likelihood:
  soliket.LensingLikelihood:
    data_folder: LensingLikelihood/
    data_filename: soliket_lensing/smo

In [129]:
model_lensing.loglike()[0]
model_lensing.loglikes()

(array([343.33859468]), [2.101003120287951e-09])

In [130]:
delta = model_lensing.likelihood["soliket.LensingLikelihood"]._get_theory(**cosmo_params)-model_lensing.likelihood["soliket.LensingLikelihood"]._get_data()[1]
-0.5 * np.dot(delta, np.linalg.inv(model_lensing.likelihood["soliket.LensingLikelihood"]._get_cov()).dot(delta))

-0.0

In [148]:
namedir=packages_path+"data/soliket_mflike/smooth_data/split_mflike/"
pre = "data_sacc_"
info_combo = {
    "likelihood": {
        "soliket.LensingLikelihood": {
            "data_filename": "soliket_lensing/smooth_data/clkk_reconstruction_sim_smooth.fits",
            "theory_lmax": 5000,
            "stop_at_error": False},
        "mflike.TTTEEE": {
            "data_folder": namedir,
            "input_file": pre + "smooth_00000.fits",
            "cov_Bbl_file": packages_path+"data/MFLike/v0.8/data_sacc_w_covar_and_Bbl.fits",
            "defaults": {"polarizations": ['TT', 'TE', 'ET', 'EE'],
                         "scales": {"TT": [30, 9000],
                                    "TE": [30, 9000],
                                    "ET": [30, 9000],
                                    "EE": [30, 9000]},
                         "symmetrize": False
                        }
        },
    },
    "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                       "WantTransfer": True,
                                       #"max_l": 9701,
                                       #"max_eta_k": 24252.5,
                                       "Transfer.high_precision": True,
                                       "Transfer.kmax": 1.2,
                                       "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3 computed according to oscilation measurements
                                       "num_nu_massive": 2, 
                                       "nu_mass_eigenstates": 2, 
                                       "nu_mass_fractions": [f2,f3],
                                       "nu_mass_numbers": [1,1],
                                       "share_delta_neff": False},
                        "stop_at_error": True}},
    "params": cosmo_params | common_nuis_params | TT_nuis_params | TE_nuis_params | EE_nuis_params,
    "packages_path": packages_path,
    "debug": True,
}

In [149]:
info_combo["theory"]["mflike.BandpowerForeground"] = {'stop_at_error': True}

In [150]:
model_combo = get_model(info_combo)

 2025-05-07 14:57:12,966 [input] 'mflike.BandpowerForeground' could not be found as internal, trying external.
 2025-05-07 14:57:12,981 [input] 'soliket.LensingLikelihood' could not be found as internal, trying external.
 2025-05-07 14:57:12,982 [input] 'mflike.TTTEEE' could not be found as internal, trying external.
 2025-05-07 14:57:13,001 [model] Input info updated with defaults (dumped to YAML):
theory:
  camb:
    version: null
    path: null
    speed: 0.3
    stop_at_error: true
    extra_args:
      lens_potential_accuracy: 1
      WantTransfer: true
      Transfer.high_precision: true
      Transfer.kmax: 1.2
      num_nu_massless: 1.044
      num_nu_massive: 2
      nu_mass_eigenstates: 2
      nu_mass_fractions:
      - 0.14763410387308012
      - 0.8523658961269198
      nu_mass_numbers:
      - 1
      - 1
      share_delta_neff: false
    ignore_obsolete: false
    use_renames: false
    external_primordial_pk: false
    renames:
      omegabh2: ombh2
      omegach2: omch

First compare the loglike

In [151]:
#lensing loglike from multi
model_multi.components[0].likelihoods[1].logp()

343.3281707269375

In [152]:
#lensing loglike from LensingLikelihood only
model_lensing.loglike()[0]

343.3385946813684

In [154]:
#lensing loglike from hand-made combo
model_combo.loglikes()[0][0]

343.3281707269375

Loglike from multi and hand-made combo match, but they do not match with lensing only. Checking CAMBdata() params

In [155]:
#multi
model_multi.provider.requirement_providers["CAMB_transfers"].current_state["results"][0]

class: <CAMBparams>
 WantCls = True
 WantTransfer = True
 WantScalars = True
 WantTensors = False
 WantVectors = False
 WantDerivedParameters = True
 Want_cl_2D_array = False
 Want_CMB = True
 Want_CMB_lensing = True
 DoLensing = True
 NonLinear = NonLinear_lens
 Transfer: <TransferParams>
   high_precision = True
   accurate_massive_neutrinos = False
   kmax = 1.2
   k_per_logint = 0
   PK_num_redshifts = 1
   PK_redshifts = [0.0]
 want_zstar = False
 want_zdrag = False
 min_l = 2
 max_l = 9151
 max_l_tensor = 600
 max_eta_k = 22877.5
 max_eta_k_tensor = 1200.0
 ombh2 = 0.02236
 omch2 = 0.1202
 omk = 0.0
 omnuh2 = 0.0006304672808897119
 H0 = 67.26717968167824
 TCMB = 2.7255
 YHe = 0.24585121447518005
 num_nu_massless = 1.044
 num_nu_massive = 2
 nu_mass_eigenstates = 2
 share_delta_neff = False
 nu_mass_degeneracies = [1.0146666666666666, 0.0]
 nu_mass_fractions = [0.14763410387308012, 0.8523658961269198]
 nu_mass_numbers = [1, 1]
 InitPower: <InitialPowerLaw>
   tensor_parameterizati

In [156]:
#lensing only
model_lensing.provider.requirement_providers["CAMB_transfers"].current_state["results"][0]

class: <CAMBparams>
 WantCls = True
 WantTransfer = True
 WantScalars = True
 WantTensors = False
 WantVectors = False
 WantDerivedParameters = True
 Want_cl_2D_array = False
 Want_CMB = True
 Want_CMB_lensing = True
 DoLensing = True
 NonLinear = NonLinear_lens
 Transfer: <TransferParams>
   high_precision = True
   accurate_massive_neutrinos = False
   kmax = 1.2
   k_per_logint = 0
   PK_num_redshifts = 1
   PK_redshifts = [0.0]
 want_zstar = False
 want_zdrag = False
 min_l = 2
 max_l = 5150
 max_l_tensor = 600
 max_eta_k = 18000.0
 max_eta_k_tensor = 1200.0
 ombh2 = 0.02236
 omch2 = 0.1202
 omk = 0.0
 omnuh2 = 0.0006304672808897119
 H0 = 67.26717968167824
 TCMB = 2.7255
 YHe = 0.24585121447518005
 num_nu_massless = 1.044
 num_nu_massive = 2
 nu_mass_eigenstates = 2
 share_delta_neff = False
 nu_mass_degeneracies = [1.0146666666666666, 0.0]
 nu_mass_fractions = [0.14763410387308012, 0.8523658961269198]
 nu_mass_numbers = [1, 1]
 InitPower: <InitialPowerLaw>
   tensor_parameterizati

In [157]:
#hand-made combo
model_combo.provider.requirement_providers["CAMB_transfers"].current_state["results"][0]

class: <CAMBparams>
 WantCls = True
 WantTransfer = True
 WantScalars = True
 WantTensors = False
 WantVectors = False
 WantDerivedParameters = True
 Want_cl_2D_array = False
 Want_CMB = True
 Want_CMB_lensing = True
 DoLensing = True
 NonLinear = NonLinear_lens
 Transfer: <TransferParams>
   high_precision = True
   accurate_massive_neutrinos = False
   kmax = 1.2
   k_per_logint = 0
   PK_num_redshifts = 1
   PK_redshifts = [0.0]
 want_zstar = False
 want_zdrag = False
 min_l = 2
 max_l = 9151
 max_l_tensor = 600
 max_eta_k = 22877.5
 max_eta_k_tensor = 1200.0
 ombh2 = 0.02236
 omch2 = 0.1202
 omk = 0.0
 omnuh2 = 0.0006304672808897119
 H0 = 67.26717968167824
 TCMB = 2.7255
 YHe = 0.24585121447518005
 num_nu_massless = 1.044
 num_nu_massive = 2
 nu_mass_eigenstates = 2
 share_delta_neff = False
 nu_mass_degeneracies = [1.0146666666666666, 0.0]
 nu_mass_fractions = [0.14763410387308012, 0.8523658961269198]
 nu_mass_numbers = [1, 1]
 InitPower: <InitialPowerLaw>
   tensor_parameterizati

max_l and max_eta_k do not match between multi/combo and lensing only. Checking if forcing lensing only to compute theory with same max_l and max_eta_k as multi/combo results in same chi2 of multi/combo

In [162]:
info_lensing_maxlmaxeta = {
    "likelihood": {
        "soliket.LensingLikelihood": {
            "data_filename": "soliket_lensing/smooth_data/clkk_reconstruction_sim_smooth.fits",
            "theory_lmax": 5000,
            "stop_at_error": False}
    },
    "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                       "WantTransfer": True,
                                       "max_l": 9151,
                                       "max_eta_k": 22877.5,
                                       "Transfer.high_precision": True,
                                       "Transfer.kmax": 1.2,
                                       "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3 computed according to oscilation measurements
                                       "num_nu_massive": 2, 
                                       "nu_mass_eigenstates": 2, 
                                       "nu_mass_fractions": [f2,f3],
                                       "nu_mass_numbers": [1,1],
                                       "share_delta_neff": False},
                        "stop_at_error": True}},
    "params": cosmo_params,
    "packages_path": packages_path,
    "debug": True,
}

In [163]:
model_lensing_maxlmaxeta = get_model(info_lensing_maxlmaxeta)

 2025-05-07 14:59:09,449 [input] 'soliket.LensingLikelihood' could not be found as internal, trying external.
 2025-05-07 14:59:09,455 [model] Input info updated with defaults (dumped to YAML):
theory:
  camb:
    version: null
    path: null
    speed: 0.3
    stop_at_error: true
    extra_args:
      lens_potential_accuracy: 1
      WantTransfer: true
      max_l: 9151
      max_eta_k: 22877.5
      Transfer.high_precision: true
      Transfer.kmax: 1.2
      num_nu_massless: 1.044
      num_nu_massive: 2
      nu_mass_eigenstates: 2
      nu_mass_fractions:
      - 0.14763410387308012
      - 0.8523658961269198
      nu_mass_numbers:
      - 1
      - 1
      share_delta_neff: false
    ignore_obsolete: false
    use_renames: false
    external_primordial_pk: false
    renames:
      omegabh2: ombh2
      omegach2: omch2
      omegal: omega_de
      omegak: omk
      yhe: YHe
      yheused: YHe
      YpBBN: Y_p
likelihood:
  soliket.LensingLikelihood:
    data_folder: LensingLikelih

Recomputing chi2s

In [164]:
#lensing only
print(model_lensing.loglike()[0])
#lensing from multi
print(model_multi.components[0].likelihoods[1].logp())
#lensing from hand-made combo
print(model_combo.loglikes()[0][0])
#lensing from lensing only with modified max_l max_eta_k
print(model_lensing_maxlmaxeta.loglike()[0])

343.3385946813684
343.3281707269375
343.3281707269375
343.3281707269375


-1751.6813269158292

In [10]:
namedir=packages_path+"data/MFLike/v0.8/"
pre = "LAT_simu_sacc_"

info_TT = {"likelihood": {
           "mflike.TT": {
                    "input_file": pre + "00000.fits",
                    "cov_Bbl_file": "data_sacc_w_covar_and_Bbl.fits",
                    #"data": {
                    #    "experiments": ["LAT_93"],
                    #    "spectra": [{"experiments": ["LAT_93", "LAT_93"]}]
                    #}
                },
},
    "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                       #"WantTransfer": True,
                                       #"lmax": 9001,
                                       #"Transfer.high_precision": True,
                                       #"Transfer.kmax": 1.2,
                                       #"max_l": 9701,
                                       #"max_eta_k": 24252.5,
                                       #"max_l" : 5150,
                                       #"max_eta_k" : 18000.0,
                                       #"num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3 computed according to oscilation measurements
                                       #"num_nu_massive": 2, 
                                       #"nu_mass_eigenstates": 2, 
                                       #"nu_mass_fractions": [f2,f3],
                                       #"nu_mass_numbers": [1,1],
                                       #"share_delta_neff": True
                                      },
                        "stop_at_error": True}},
    "params": cosmo_params | nuis_params,
    "packages_path": packages_path,
    "debug": True,
}
info_TT["theory"]["mflike.BandpowerForeground"] = {'requested_cls': ['tt'],
                                                    'stop_at_error': True
                                                  }

In [11]:
model_TT = get_model(info_TT)
model_TT.loglikes()
#TT=model_mTT.components[0].likelihoods[0]

camb {} in input.py
 2025-04-16 12:03:27,658 [input] 'mflike.BandpowerForeground' could not be found as internal, trying external.
mflike.BandpowerForeground {'beta_s': {'value': -2.5, 'latex': '\\beta_s'}, 'alpha_s': {'value': 1.0, 'latex': '\\alpha_s'}, 'T_effd': {'value': 19.6, 'latex': 'T_{\\mathrm{dust},\\mathrm{eff}}'}, 'beta_d': {'value': 1.5, 'latex': '\\beta_\\mathrm{dust}'}, 'a_tSZ': {'prior': {'min': 3.0, 'max': 3.6}, 'proposal': 0.05, 'latex': 'a_\\mathrm{tSZ}'}, 'a_kSZ': {'prior': {'min': 1.4, 'max': 1.8}, 'proposal': 0.1, 'latex': 'a_\\mathrm{kSZ}'}, 'a_p': {'prior': {'min': 6.2, 'max': 7.6}, 'proposal': 0.075, 'latex': 'a_p'}, 'beta_p': {'prior': {'min': 1.8, 'max': 2.2}, 'proposal': 0.015, 'latex': '\\beta_p'}, 'a_c': {'prior': {'min': 4.4, 'max': 5.4}, 'proposal': 0.12, 'latex': 'a_c'}, 'beta_c': {'prior': {'min': 2.0, 'max': 2.4}, 'proposal': 0.03, 'latex': '\\beta_c'}, 'a_s': {'prior': {'min': 2.8, 'max': 3.4}, 'proposal': 0.01, 'latex': 'a_s'}, 'a_gtt': {'prior': {'

(array([-1751.68132692]), [2.101003120287951e-09])

In [6]:
a={}
a["comp"] = {"mflike":{},"lens":{}}
for k,kk in a["comp"].items():
    print(k,kk)

print([kv for kv in a["comp"].items()])

mflike {}
lens {}
[('mflike', {}), ('lens', {})]


{'components': ['mflike.TTTEEE', 'soliket.LensingLikelihood'], 'options': [{'input_file': 'LAT_simu_sacc_00044.fits', 'cov_Bbl_file': 'data_sacc_w_covar_and_Bbl.fits', 'stop_at_error': True, 'data': {'experiments': ['LAT_93'], 'spectra': [{'experiments': ['LAT_93', 'LAT_93']}]}}, {'theory_lmax': 5000}], 'stop_at_error': True} getdef
{'requested_cls': ['tt', 'te', 'ee'], 'defaults': {'polarizations': ['TT', 'TE', 'ET', 'EE'], 'scales': {'TT': [30, 9000], 'TE': [30, 9000], 'ET': [30, 9000], 'EE': [30, 9000]}, 'symmetrize': False}, 'data_folder': 'LensingLikelihood/', 'input_file': None, 'cov_Bbl_file': None, 'lmax_theory': None, 'data': {'experiments': ['LAT_93', 'LAT_145', 'LAT_225'], 'spectra': [{'experiments': ['LAT_93', 'LAT_93']}, {'experiments': ['LAT_93', 'LAT_145']}, {'experiments': ['LAT_93', 'LAT_225']}, {'experiments': ['LAT_145', 'LAT_145']}, {'experiments': ['LAT_145', 'LAT_225']}, {'experiments': ['LAT_225', 'LAT_225']}]}, 'systematics_template': None, 'type': [], 'speed': 

In [1]:
fg_values_a = {"a_tSZ": TT_nuis_params["a_tSZ"], "a_kSZ": TT_nuis_params["a_kSZ"]}
fg_values_b = {k: v * 1.1 for k, v in fg_values_a.items()}
logp_a = model.loglikes(fg_values_a, cached=False)[0].sum()
logp_b = model.loglikes(fg_values_b, cached=False)[0].sum()
d_logp = logp_b - logp_a
#assert np.isclose(d_logp, -503.395, rtol=1e-4)

NameError: name 'TT_nuis_params' is not defined

In [160]:
model1_logp_a = model1.loglikes(fg_values_a, cached=False)[0].sum()
model2_logp_a = model2.loglikes({}, cached=False)[0].sum()
model1_logp_b = model1.loglikes(fg_values_b, cached=False)[0].sum()
model2_logp_b = model2.loglikes({}, cached=False)[0].sum()
d_logp1 = model1_logp_b - model1_logp_a
d_logp2 = model2_logp_b - model2_logp_a
d_logp_sum = d_logp1 + d_logp2
assert np.isclose(d_logp, d_logp_sum, rtol=1e-5)